## Essential code to build neural networks using TensorFlow

1. To initialize a simple sequential model we use $\texttt{tf.keras.models.Sequential}$.
1. To implement a simple linear transformation with some activation we use $\texttt{tf.keras.models.layers.Dense}$.
1. After building a model, we need to **compile** it, where we use the method $\texttt{.compile}$. This is essentially to tell that we are "finished" with initializing our model, so it may be considered a part of the initialization process.
1. Training is exactly like with $\texttt{sklearn}$, i.e. we use the method $\texttt{.fit}$.
1. We can also predict much like as with $\texttt{sklearn}$, i.e. using the method $\texttt{.predict}$. However, to evaluate our model, it is easier to use the method $\texttt{.evaluate}$.
1. We can take a look at our model using the method $\texttt{.summary}$, which is super cool and useful!

**Note**: There are more advanced (but also more flexible) ways to initialize and train a model. However, the above "steps" are useful to solve most problems.

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
print(load_breast_cancer().DESCR)

.. _breast_cancer_dataset:

Breast cancer Wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

:Number of Instances: 569

:Number of Attributes: 30 numeric, predictive attributes and the class

:Attribute Information:
    - radius (mean of distances from center to points on the perimeter)
    - texture (standard deviation of gray-scale values)
    - perimeter
    - area
    - smoothness (local variation in radius lengths)
    - compactness (perimeter^2 / area - 1.0)
    - concavity (severity of concave portions of the contour)
    - concave points (number of concave portions of the contour)
    - symmetry
    - fractal dimension ("coastline approximation" - 1)

    The mean, standard error, and "worst" or largest (mean of the three
    worst/largest values) of these features were computed for each image,
    resulting in 30 features.  For instance, field 0 is Mean Radius, field
    10 is Radius SE, field 20 is Worst Radius.

    - 

In [1]:
#@title Breast Cancer Classification with NN
import tensorflow as tf

# Use the `load_breast_cancer` function to construct your dataset
x, y = load_breast_cancer(return_X_y=True)

# Use `train_test_split` to split your data into a train and a test set.
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

scaler = StandardScaler()
z_train = scaler.fit_transform(x_train)
z_test = scaler.transform(x_test)

our_first_nn = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(z_train.shape[1],)),
    # input shape required in the first layer
    tf.keras.layers.Dense(64, activation='relu'),
    # map 64 to 128 features and apply ReLU
    tf.keras.layers.Dense(128, activation='relu'),
    # softmax is used for classification
    tf.keras.layers.Dense(2, activation='softmax'),
    ])

our_first_nn.summary()


: 

In [ ]:
our_first_nn.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

our_first_nn.fit(z_train, y_train, epochs=3)

loss, accuracy = our_first_nn.evaluate(z_test, y_test)

print(
    f'\nOut first neural network managed {round(accuracy * 100, 2)}% accuracy.'
)

In [ ]:
#@title Classification Example (MNIST Digits)
import tensorflow as tf
from tensorflow.keras import layers, models

# Load dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize (scale 0-255 pixel values to 0-1)
X_train, X_test = X_train / 255.0, X_test / 255.0

# Flatten images (28x28 → 784)
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

# Build model
model = models.Sequential([
    layers.Input(shape=(784,)),      # Explicit input layer
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train,
                    epochs=5,
                    batch_size=32,
                    validation_data=(X_test, y_test))

# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.3f}")


In [ ]:
# Plot training & validation accuracy
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title("Training vs Validation Accuracy")
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title("Training vs Validation Loss")
plt.show()

In [ ]:
from tensorflow.keras.utils import plot_model

# Assume `model` is your Keras model
plot_model(model, show_shapes=True, show_layer_names=True, to_file="model.png")

In [ ]:
#@title Regression
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
california = fetch_california_housing()
X, y = california.data, california.target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape)

In [ ]:
# Build model with Input layer
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # Explicit input layer
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae'])

history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=16,
                    validation_data=(X_test, y_test))

test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae:.3f}")

In [ ]:
# Assume `model` is your Keras model
plot_model(model, show_shapes=True, show_layer_names=True, to_file="model.png")

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation MAE
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.title("Training vs Validation MAE")
plt.show()

# Plot training & validation loss (MSE)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.legend()
plt.title("Training vs Validation Loss")
plt.show()

## **Task 1**: Binary Classification with SVM (Banknote Authentication)
![](https://www.neuraldesigner.com/images/banknote-authentication.webp)
[Image Source](https://www.neuraldesigner.com/blog/banknote-authentication/)

- **Dataset**: Banknote Authentication — wavelet-transformed images of banknotes.
- **Target**: Predict whether a banknote is `genuine` (1) or `fake` (0).

**Use the file "banknotes.csv"**

- Train a classification model using NN